In [88]:
import pandas as pd
import math

In [89]:
def ocpf(ocpf_name):
    ocpf_df = pd.read_csv(ocpf_name)
    ocpf_df = ocpf_df[ocpf_df.State.isin(['MA'])][['Date', 'City', 'Amount']]
    ocpf_df['Amount'] = ocpf_df['Amount'].replace('[\$]','', regex=True).replace('[\,]','', regex=True).astype(float)

    ocpf_df = ocpf_df.groupby(['City'])['Amount'].apply(sum).reset_index()

    # ocpf_df.to_csv("ocpf_summaries.csv")

    ocpf_df['City'] = ocpf_df['City'].map(lambda x: str(x).lower())
    ocpf_df = ocpf_df.groupby(['City'])['Amount'].apply(sum).reset_index()
    return ocpf_df

In [90]:
def senate_ocpf(ocpf_name, number):
    ocpf_df = ocpf(ocpf_name)
    #################### 
    # Senate districts #
    ####################

    senate_df = pd.read_csv("../cleaned_data/District_Mappings/senate_districts_mapped_final.csv", names=['city_town', 'senate_district', 'n/a'], header=None)[['city_town', 'senate_district']]

    senate_df['city_town'] = senate_df['city_town'].apply(lambda x: str(x).split(" -")[0]).map(lambda x: x.lower())
    senate_df['city_town'] = senate_df['city_town'].replace('[\:]','', regex=True).str.strip()
    senate_df['senate_district'] = senate_df['senate_district'].replace('(and)','&', regex=True)  
    senate_df = senate_df.groupby(['city_town'])['senate_district'].apply(list).reset_index()
    senate_df.to_csv("../cleaned_data/District_Mappings/senate_districts_mapped_to_lists_" + number + ".csv")


    senate_ocpf = pd.concat([ocpf_df.set_index('City'), senate_df.set_index('city_town')], axis=1, join='inner')
    senate_ocpf.index.name = 'city_town'
    print(senate_ocpf)
    senate_ocpf.to_csv("../cleaned_data/Contribution/contribution_senate_" + number + ".csv")

    return senate_ocpf


In [91]:
def house_ocpf(ocpf_name, number):
    #################### 
    # House districts #
    ####################
    ocpf_df = ocpf(ocpf_name)
    house_df = pd.read_csv("../cleaned_data/District_Mappings/house_districts_mapped_final.csv", names=['city_town', 'house_district', 'n/a'], header=None)[['city_town', 'house_district']]

    house_df['city_town'] = house_df['city_town'].apply(lambda x: str(x).split(" -")[0]).apply(lambda x: str(x).split(" (")[0]).map(lambda x: x.lower())
    house_df['city_town'] = house_df['city_town'].replace('[\:]','', regex=True).str.strip()
    house_districts = list(house_df['house_district'])
    ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(math.floor(n/10)%10!=1)*(n%10<4)*n%10::4])
    dumb_list = [ordinal(n) for n in range(0,38)]

    for i in range(len(house_districts)):
        splitter = house_districts[i].split(" ")
        if splitter[0] == 'Barnstable,':
            continue
        if splitter[0] == 'First':
            splitter[0] = dumb_list[1]
        elif splitter[0] == 'Second':
            splitter[0] = dumb_list[2]
        elif splitter[0] == 'Third':
            splitter[0] = dumb_list[3]
        elif splitter[0] == 'Fourth':
            splitter[0] = dumb_list[4]
        elif splitter[0] == 'Fifth':
            splitter[0] = dumb_list[5]
        elif splitter[0] == 'Sixth':
            splitter[0] = dumb_list[6]
        elif splitter[0] == 'Seventh':
            splitter[0] = dumb_list[7]
        elif splitter[0] == 'Eighth':
            splitter[0] = dumb_list[8]
        elif splitter[0] == 'Eigth':
            splitter[0] = dumb_list[8]
        elif splitter[0] == 'Ninth':
            splitter[0] = dumb_list[9]
        elif splitter[0] == 'Tenth':
            splitter[0] = dumb_list[10]
        elif splitter[0] == 'Eleventh':
            splitter[0] = dumb_list[11]
        elif splitter[0] == 'Twelfth':
            splitter[0] = dumb_list[12]
        elif splitter[0] == 'Thirteenth':
            splitter[0] = dumb_list[13]
        elif splitter[0] == 'Fourteenth':
            splitter[0] = dumb_list[14]
        elif splitter[0] == 'Fifteenth':
            splitter[0] = dumb_list[15]
        elif splitter[0] == 'Sixteenth':
            splitter[0] = dumb_list[16]
        elif splitter[0] == 'Seventeenth':
            splitter[0] = dumb_list[17]
        elif splitter[0] == 'Eighteenth':
            splitter[0] = dumb_list[18]
        elif splitter[0] == 'Nineteenth':
            splitter[0] = dumb_list[19]
        elif splitter[0] == 'Ninteenth':
            splitter[0] = dumb_list[19]
        elif splitter[0] == 'Twentieth':
            splitter[0] = dumb_list[20]
        elif splitter[0] == 'Twenty-First':
            splitter[0] = dumb_list[21]
        elif splitter[0] == 'Twenty-Second':
            splitter[0] = dumb_list[22]
        elif splitter[0] == 'Twenty-Third':
            splitter[0] = dumb_list[23]
        elif splitter[0] == 'Twenty-Fourth':
            splitter[0] = dumb_list[24]
        elif splitter[0] == 'Twenty-Fifth':
            splitter[0] = dumb_list[25]
        elif splitter[0] == 'Twenty-Sixth':
            splitter[0] = dumb_list[26]
        elif splitter[0] == 'Twenty-Seventh':
            splitter[0] = dumb_list[27]
        elif splitter[0] == 'Twenty-Eighth':
            splitter[0] = dumb_list[28]
        elif splitter[0] == 'Twenty-Ninth':
            splitter[0] = dumb_list[29]
        elif splitter[0] == 'Thirtieth':
            splitter[0] = dumb_list[30]
        elif splitter[0] == 'Thirty-First':
            splitter[0] = dumb_list[31]
        elif splitter[0] == 'Thirty-Second':
            splitter[0] = dumb_list[32]
        elif splitter[0] == 'Thirty-Third':
            splitter[0] = dumb_list[33]
        elif splitter[0] == 'Thirty-Fourth':
            splitter[0] = dumb_list[34]
        elif splitter[0] == 'Thirty-Fifth':
            splitter[0] = dumb_list[35]
        elif splitter[0] == 'Thirty-Sixth':
            splitter[0] = dumb_list[36]
        elif splitter[0] == 'Thirty-Seventh':
            splitter[0] = dumb_list[37]

        district = ''
        for j in range(len(splitter)):
            if j != len(splitter)-1:
                district += splitter[j]+" "
            else:
                district += splitter[j]
        house_districts[i] = district

    house_df['house_district'] = house_districts
    house_df = house_df.groupby(['city_town'])['house_district'].apply(list).reset_index()
    house_df.to_csv("../cleaned_data/District_Mappings/house_districts_mapped_to_lists_" + number + ".csv")
    house_ocpf = pd.concat([ocpf_df.set_index('City'), house_df.set_index('city_town')], axis=1, join='inner')
    house_ocpf.index.name = 'city_town'
    print(house_ocpf)
    house_ocpf.to_csv("../cleaned_data/Contribution/contribution_data_house_districts_" + number + ".csv")
    return house_ocpf


In [92]:
#########
# 2010 #
########
senate_2010 = senate_ocpf("../original_data/ocpf-output-2010.csv", "2010")
house_2010 = house_ocpf("../original_data/ocpf-output-2010.csv", "2010")

                       Amount  \
city_town                       
abington             10000.00   
acton                34000.00   
adams                15300.10   
agawam               52973.71   
amesbury              2000.00   
amherst               6200.00   
andover             214248.50   
arlington            20693.56   
ashburnham            9192.68   
ashland               4000.00   
athol                22490.00   
attleboro            27500.36   
auburn               90425.00   
avon                158152.00   
ayer                 10468.01   
barnstable           17900.87   
barre                 1200.00   
becket               10000.00   
bedford              23000.00   
belmont             114901.99   
beverly              73520.93   
billerica            12572.00   
blandford             1000.00   
bolton               15300.00   
boston            12391098.29   
boxborough            3000.00   
boxford               2000.00   
boylston              3500.00   
braintree 

In [ ]:
#########
# 2011 #
########
senate_2010 = senate_ocpf("../original_data/ocpf-output-2011.csv", "2011")
house_2010 = house_ocpf("../original_data/ocpf-output-2011.csv", "2011")

In [ ]:
#########
# 2012 #
########
senate_2010 = senate_ocpf("../original_data/ocpf-output-2012.csv", "2012")
house_2010 = house_ocpf("../original_data/ocpf-output-2012.csv", "2012")

In [ ]:
#########
# 2013 #
########
senate_2010 = senate_ocpf("../original_data/ocpf-output-2013.csv", "2013")
house_2010 = house_ocpf("../original_data/ocpf-output-2013.csv", "2013")

In [46]:
#########
# 2014 #
########
senate_2010 = senate_ocpf("../original_data/ocpf-output-2014.csv", "2014")
house_2010 = house_ocpf("../original_data/ocpf-output-2014.csv", "2014")

In [ ]:
#########
# 2015 #
########
senate_2010 = senate_ocpf("../original_data/ocpf-output-2015.csv", "2015")
house_2010 = house_ocpf("../original_data/ocpf-output-2015.csv", "2015")

In [ ]:
#########
# 2016 #
########
senate_2010 = senate_ocpf("../original_data/ocpf-output-2016.csv", "2016")
house_2010 = house_ocpf("../original_data/ocpf-output-2016.csv", "2016")

In [ ]:
#########
# 2017 #
########
senate_2010 = senate_ocpf("../original_data/ocpf-output-2017.csv", "2017")
house_2010 = house_ocpf("../original_data/ocpf-output-2017.csv", "2017")